# Dieses Notebook macht die Prognosen für die nächsten 5 Jahre

## Prognose der durchschnittlichen Sonnenstunden mit Konfidenzintervall

In [2]:
# Bibliotheken importieren
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import plotly.graph_objects as go

# Dateipfade für die CSV-Dateien definieren
file_path_sun_data = '../data/meteo/meteo_swiss_data.csv'
file_path_stn_data = '../data/meteo/meteo_swiss_stn_data.csv'

# Daten laden
sun_data = pd.read_csv(file_path_sun_data, delimiter=';')
stn_data = pd.read_csv(file_path_stn_data, delimiter=';')

# Einträge für Liechtenstein entfernen
stn_data = stn_data[stn_data['canton'] != 'LI']

# Wetter- und Stationsdaten verknüpfen
data = pd.merge(sun_data, stn_data, on='stn', how='inner')

# Zeitspalte in Datumsformat konvertieren
data['time'] = pd.to_datetime(data['time'], format='%Y%m')
data['year'] = data['time'].dt.year
data['month'] = data['time'].dt.to_period('M').astype(str)

# '-' durch '0' ersetzen und in Float konvertieren
data['sunhours'] = data['su2000m0'].replace('-', '0').astype(float)

# Ungültige Werte entfernen
data = data[(data['sunhours'] > 0) & (data['year'] != 2025)]

# Monatliche Daten aggregieren
monthly_data = data.groupby(data['time'].dt.to_period('M'))['sunhours'].mean().reset_index()
monthly_data['time'] = pd.to_datetime(monthly_data['time'].astype(str), format='%Y-%m')

# Sicherstellen, dass die Zeitreihe lückenlos ist
full_date_range = pd.date_range(start=monthly_data['time'].min(), end=monthly_data['time'].max(), freq='MS')
df = monthly_data.set_index('time').reindex(full_date_range).rename_axis('time').reset_index()
df['sunhours'] = df['sunhours'].fillna(0)

# SARIMAX-Modell trainieren
model = SARIMAX(df['sunhours'], order=(1,1,1), seasonal_order=(1,1,1,12), enforce_invertibility=False)
results = model.fit()

# Prognose für 72 Monate (6 Jahre) erstellen
forecast = results.get_forecast(steps=72)
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()

# Interaktives Diagramm mit Plotly erstellen
fig = go.Figure()

# Historische Daten hinzufügen
fig.add_trace(go.Scatter(x=df['time'], y=df['sunhours'], mode='lines', name='Historische Daten', line=dict(color='#1f77b4')))

# Prognosedaten hinzufügen
forecast_index = pd.date_range(start=df['time'].iloc[-1], periods=72, freq='MS')
fig.add_trace(go.Scatter(x=forecast_index, y=forecast_mean, mode='lines', name='Prognose', line=dict(color='#ff7f0e')))

# Konfidenzintervall als schattierten Bereich hinzufügen
fig.add_trace(go.Scatter(
    x=list(forecast_index) + list(forecast_index[::-1]),
    y=list(forecast_ci.iloc[:, 1]) + list(forecast_ci.iloc[:, 0][::-1]),
    fill='toself',
    fillcolor='rgba(255, 127, 14, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo='skip',
    name='Konfidenzintervall'
))

# Layout anpassen
fig.update_layout(
    title='Prognose der durchschnittlichen Sonnenstunden mit Konfidenzintervall',
    yaxis_title='Durchschnittliche Sonnenstunden',
    template='plotly_white',
    legend=dict(font=dict(size=12)),
    xaxis=dict(
        tickformat='%Y',
        dtick='M12',  # Jährliche Ticks
        ticklabelmode='period'  # Labels zwischen den Gittern anzeigen
    )
)

# Diagramm anzeigen
fig.show()

# Diagramm als HTML-Datei speichern
fig.write_html('../docs/assets/diagramme/sunhours_forecast.html')

## Polynomiale Prognose (Grad 2) der kumulierten Leistung je Kanton

In [3]:
# Bibliotheken importieren
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import plotly.graph_objects as go

# CSV-Datei mit den PV-Anlagen laden
file_path = '../data/electricity_production_plants/ElectricityProductionPlant.csv'
df = pd.read_csv(file_path)

# Subkategorien laden und zuordnen
sub_categories = pd.read_csv('../data/electricity_production_plants/SubCategoryCatalogue.csv')
df['SubCategory'] = df['SubCategory'].replace(dict(zip(sub_categories['Catalogue_id'], sub_categories['de'])))

# Nur Photovoltaik-Anlagen filtern
df = df[df['SubCategory'] == 'Photovoltaik'].copy()

# Jahr aus 'BeginningOfOperation' extrahieren
df['Year'] = pd.to_datetime(df['BeginningOfOperation'], errors='coerce').dt.year

# Daten gruppieren und kumulativ berechnen
cumulative_data = df.groupby(['Year', 'Canton'])['TotalPower'].sum().groupby(level=1).cumsum().reset_index()
filtered_data = cumulative_data[(cumulative_data['Year'] >= 2015) & (cumulative_data['Year'] <= 2024)].copy()
filtered_data['CumulativePower'] = filtered_data.groupby('Canton')['TotalPower'].cumsum()

# Top 4 Kantone mit höchstem Endwert filtern
end_values = filtered_data.groupby('Canton')['CumulativePower'].max()
top_4_cantons = end_values.nlargest(4).index.tolist()
df = filtered_data[filtered_data['Canton'].isin(top_4_cantons)].drop(columns=['TotalPower']).copy()

# Prognosejahre definieren
years_forecast = np.arange(2024, 2031)

# Neues Diagramm erstellen
fig = go.Figure()

# Farben für die Kantone definieren
colors = {
    'BE': '#1f77b4',  # Blau für Bern
    'ZH': '#ff7f0e',  # Orange für Zürich
    'VD': '#2ca02c',  # Grün für Waadt
    'SG': '#d62728'   # Rot für St. Gallen
}

# Daten für jeden Kanton verarbeiten
for canton in df['Canton'].unique():
    df_canton = df[df['Canton'] == canton].copy()
    x = df_canton['Year'].values
    y = df_canton['CumulativePower'].values

    # Historische Werte plotten
    fig.add_trace(go.Scatter(
        x=x, y=y,
        mode='markers+lines',
        name=f'{canton} (historisch)',
        line=dict(color=colors[canton])
    ))

    try:
        # Polynom 2. Ordnung fitten
        coeffs = np.polyfit(x, y, deg=2)
        poly_func = np.poly1d(coeffs)
        y_forecast = poly_func(years_forecast)

        # Prognose plotten
        fig.add_trace(go.Scatter(
            x=years_forecast, y=y_forecast,
            mode='lines',
            line=dict(dash='dot', color=colors[canton]),
            name=f'{canton} (Prognose)'
        ))
    except Exception as e:
        print(f"{canton}: Fehler beim Fit: {e}")

# Layout des Diagramms anpassen
fig.update_layout(
    title="Polynomiale Prognose (Grad 2) der kumulierten Leistung je Kanton (2024–2030)",
    yaxis_title="Kumulative Leistung (kWp)",
    hovermode='x unified',
    xaxis=dict(
        tickformat='%Y',
        dtick=1  # Jährliche Ticks
    )
)

# Diagramm anzeigen
fig.show()

# Diagramm als HTML-Datei speichern
fig.write_html('../docs/assets/diagramme/cumulative_power_forecast.html')